# EICU data from the Dungog East Coast Low

This notebook imports and explores the cleaned up data from the EICU for the Dungug East Coast Low event. The data have been cleaned to remove empty fields, and an error in the lat/lon values for Medowie (where the lat and lon values were swapped). 

The working directory for this project is: 

*\\nas\gemd\georisk\HaRIA_B_Wind\projects\impact_forecasting\data\exposure\NSW\April_2015_Impact_Assessment*

**Date: August 2017, Creator: Claire Krause**

The data have been combined with outputs from the BoM's high resolution Dungog East Coast Low model ensemble. The netcdf outputs from the model were converted to Geotiffs using gdal_translate 

`e.g. gdal_translate -of GTiff NETCDF:combined_alltimes_accum_ls_rainrate_stage4_ens00.nc:lsrain combined_alltimes_accum_ls_rainrate_stage4_ens00.tif`

The Geotiffs were then read into ArcGIS, along side the EICU data. The model raster value at each EICU location was determined using the following code:

`import arcpy
from arcpy.sa import *
arcpy.CheckOutExtension('Spatial')
ExtractValuesToPoints("<observation_points>", "<TCRMwindfield>.tif", "<outputlayername>", "NONE", "VALUE_ONLY")`

The raster values were exported to a .txt file, and added to the final columns of the EICU data.

# Set up the notebook

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from os.path import join as pjoin
import pandas as pd
import geopandas as gpd
import numpy as np

import seaborn as sns
sns.set_context("talk")
sns.set_style("whitegrid")
sns.set_palette("hls")

# Start by reading in an exploring the dataset

In [ ]:
data_path = "X:/georisk/HaRIA_B_Wind/projects/impact_forecasting/data/impact/dungog"
filename = "damage_hazard.shp"
filepath = pjoin(data_path, filename)

df = gpd.read_file(filepath)
df

### List of column headings

In [ ]:
df.columns

# Just do some quick cleaning up of the data

There are a number of blank fields throughout the data where a value was not entered into the dataset by the assessor. We need to keep track of the missing data, as well as the entered data, so we will find all 'NaN' values in the dataset, and change these to 'Not given' so we can include them in subsequent analyses. 

In [ ]:
df = df.fillna('Not given')

### How many entries do we have for each column?
I.e. not NaN. Note that this does not check the contents of the field, only that something was written into it

NB: Something isn't quite working here. Use the values listed in the text below, not the outputs from this cell

In [ ]:
df.count()

# EICU data for the Hunter Region ECL event:

* Number of data entries = 2072
* Lat/lon = 2062 (99.5%)
* EICU_Degdamage - damage state rank = 2055 (99.2%)
* Insurance state = 1821 (88%)
* Building use = 1909 (92%)
* Foundations / Structure / Roof type = 1821 (88%)
* Water inundation = 1317 (64%)
* Water level = 203 (10%)

# Lets explore some key fields in more detail...

In [ ]:
damage = df.groupby('EICU_Degda').count()
damage.OBJECTID

## Damage and water?

In [ ]:
order = ['No Damage - 0%', 'Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%', 'Destroyed - 76-100%']
ax = sns.countplot(x="EICU_Degda", hue="waterinund", data=df, order=order)
ax.legend(title="Water Damage?", loc=1)
ax.set_yscale("log", nonposy='clip')
plt.xticks(rotation=10)
plt.title('Damage State and presence of water')
plt.xlabel('Damage state')

There does seem to be some relationship between the presence of water and damage state. Note that the last two columns represent missing vales, and where the surveyor has selected 'not applicable'.

## Damage by roof type

In [ ]:
temp = []
for name, group in df.groupby('EICU_Degda'):
    numingroup = group['OBJECTID'].count()
    numroofs = group.groupby('roof').count()
    percroofs = numroofs['OBJECTID']/numingroup*100
    for keys in percroofs.keys():
        temp.append((name, keys, percroofs[keys]))

roofdamage = pd.DataFrame(temp, columns = ['Damage_state', 'Roof_type', 'Percent'])
roofdamage

In [ ]:
order = ['No Damage - 0%', 'Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%', 'Destroyed - 76-100%']
ax = sns.factorplot(x="Damage_state", y = 'Percent', hue="Roof_type", data=roofdamage, order=order, kind = 'bar', size = 10)
plt.xticks(rotation=10)
plt.title('Damage state and roof type')
plt.ylabel('Percent of damage category')
plt.xlabel('Damage state')

## Damage by wall structure type

In [ ]:
temp = []
for name, group in df.groupby('EICU_Degda'):
    numingroup = group['OBJECTID'].count()
    numroofs = group.groupby('structure_').count()
    percroofs = numroofs['OBJECTID']/numingroup*100
    for keys in percroofs.keys():
        temp.append((name, keys, percroofs[keys]))

structuredamage = pd.DataFrame(temp, columns = ['Damage_state', 'Structure_type', 'Percent'])
structuredamage

In [ ]:
order = ['No Damage - 0%', 'Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%', 'Destroyed - 76-100%']
ax = sns.factorplot(x="Damage_state", y = 'Percent', hue="Structure_type", data=structuredamage, order=order, 
                    kind = 'bar', size = 10)
plt.xticks(rotation=10)
plt.title('Damage state and structure type')
plt.ylabel('Percent of damage category')
plt.xlabel('Damage state')

# Can we find wind damage specifically?

In [ ]:
order = ['No Damage - 0%', 'Minor Impact - 1-25%', 'Major Impact - 26-50%', 'Severe Impact - 51-75%', 'Destroyed - 76-100%']
ax = sns.countplot(x="EICU_Degda", hue="eventtype", data=df, order=order)
ax.legend(title="Damage event", loc=1)
ax.set_yscale("log", nonposy='clip')
plt.xticks(rotation=10)
plt.title('Damage States for flood vs storm')
plt.xlabel('Damage state')

# <font color=red>Question for follow up with NSWFR</font>

It may be possible to distinguish between flood and storm damage, however it is not clear whether 'flood' damage refers specifically to riverine flooding, or whether water damage from a leaking roof is also classified as flood. 

In [ ]:
order = ['Stable', 'Minor', 'Moderate', 'Severe']
ax = sns.countplot(x="roofdam", hue="waterinund", data=df, order = order)
ax.legend(title="Water Damage?")
ax.set_yscale("log", nonposy='clip')
plt.xticks(rotation=10)
plt.title('Roof damage State and presence of water')
plt.xlabel('Roof damage state')

In [ ]:
order = ['Stable', 'Minor', 'Moderate', 'Severe']
ax = sns.countplot(x="roofdam", hue="eventtype", data=df, order = order)
ax.legend(title="Event type")
ax.set_yscale("log", nonposy='clip')
plt.xticks(rotation=10)
plt.title('Roof damage State and presence of water')
plt.xlabel('Roof damage state')

No building with roof damage state listed attributes the damage to storm. This suggests that 'flood' is being used to describe any type of water damage, whether it is rising water, or water coming into the house another way. 

This is problematic, as it means it isn't possible to use the 'event type' tag to distinguish between these two scenarios.

## NB
Wind damage doesn't seem to be discernable from this dataset. Roof damage could be used a pseudo-proxy for storm damage, however it is not specifically marked as storm damage in the database. For all of the entries that have roof damage assessed, all of them list the event type as 'flood', not 'storm'. This suggests that water ingress from damaged roofs is classed as flood damage, not storm damage for this event.

# Can we generate a vulnerability-like curve of damage state and modelled wind?

Wind speed was output at each model time step in the ECL event ensemble simulations. These plots use the maximum wind speed over the all time steps for each grid point.

In the first instance, two individual ensemble members (randomly chosen) were interrogated. It was found that these did not differ greatly, and so additional members were not further investigated.

## The model outputs sustained wind speed. We would like to convert this to wind gust.

To do this, we applied a gust conversion factor of 1.6 to see if this improved the results.

In [ ]:
#df['combined_alltimes_maxwind_stage4_ens12'] = df['combined_alltimes_maxwind_stage4_ens12'] * 1.6
#df['combined_alltimes_maxwind_stage4_ens22'] = df['combined_alltimes_maxwind_stage4_ens22'] * 1.6

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degda", x = "PSWG", data = df, order = order)
sns.swarmplot(y = "EICU_Degda", x = "PSWG", data = df, order = order, color = "0.2")
plt.xlabel("Surface gust wind speed (m/s)")
plt.ylabel("Damage state")

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degda", x = "PSWG", data = df, order = order)
sns.swarmplot(y = "EICU_Degda", x = "PSWG", data = df, order = order, color = "0.2")
plt.xlabel("Surface gust wind speed (m/s)")
plt.ylabel("Damage state")

When the gust factor is applied, there emerges a threshold-like response to wind speed at around 12 m/s (43 km/hr). The damage relationship however, doesn't increase with increasing wind. The greatest wind speeds are not experienced by the buildings that sustained the greatest damage. 

This suggests that wind may not be the driving factor in generating damage in this event. 

# How about rain?

Is it possible that rain is the factor driving the damage in this event?

The rain rate (kg m-2 s-1) was output for each model time step, and the maximum rain rate throughout the model run was extracted. Again, two random ensemble members were chosen for initial investigation.

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degda", x = "P6RR", data = df, order = order)
sns.swarmplot(y = "EICU_Degda", x = "P6RR", data = df, order = order, 
              color = "0.2")
plt.xlabel("Maximum 6-hour rainfall rate (mm/6hrs)")
plt.ylabel("Damage state")

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degda", x = "PTEA", data = df, order = order)
sns.swarmplot(y = "EICU_Degda", x = "PTEA", data = df, order = order, 
              color = "0.2")
plt.xlabel("Total accumulated precipitation (mm)")
plt.ylabel("Damage state")

The results from the two ensemble members are quite different, suggesting that this variable is much more spatially variable. To address this, we extracted the maximum rain rate for each ensemble member, then calculated the average of those rain rates for each grid box. 

## Ensemble mean rain rate over all time steps

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degdamage", x = "combined_alltimes_accum_ls_rainrate_stage4_ensMEAN", data = df, order = order)
sns.swarmplot(y = "EICU_Degdamage", x = "combined_alltimes_accum_ls_rainrate_stage4_ensMEAN", data = df, order = order, 
              color = "0.2")

## Ensemble standard deviation of rain rate over all ensemble members

In [ ]:
order = ['Destroyed - 76-100%', 'Severe Impact - 51-75%', 'Major Impact - 26-50%', 'Minor Impact - 1-25%', 'No Damage - 0%']
sns.boxplot(y = "EICU_Degdamage", x = "combined_alltimes_accum_ls_rainrate_stage4_ensSTD", data = df, order = order)
sns.swarmplot(y = "EICU_Degdamage", x = "combined_alltimes_accum_ls_rainrate_stage4_ensSTD", data = df, order = order, 
              color = "0.2")

# Results

#### The relationship between damage and rain rate looks much more convincing than with wind. This suggests that it was the rainfall that was the driver of damamge in this event. 

#### This makes sense, as the design wind speed for houses is 38-40 m/s, which is the upper limit of modelled wind speeds for building locations. It is therefore not likely that the wind speed was sufficiently high enough to break apart houses. 

#### The much more logical solution is that the rain rate caused water ingress to houses, either directly through weaknesses in the structure, or through damage to the building's exterior caused by trees falling.

# Can we find any relationship between tree damage and wind speed?

## Locations where 'dangertree' column has been ticked

In [ ]:
tree_text = df[df['dangertree'].str.contains('Yes')]
tree_text.shape

In [ ]:
sns.distplot(tree_text.combined_alltimes_maxwind_stage4_ens12)
plt.title('Histogram of wind speeds for entries with the \'dangertree\' column ticked')
plt.xlabel('Wind speed (gust factor = 1.6, m/s)')

## Look for terms relating to tree in the comments field

In [ ]:
tree_text = df[df['Comments'].str.contains('tree|branch', case = False)]
tree_text.shape

In [ ]:
sns.distplot(tree_text.combined_alltimes_maxwind_stage4_ens12)
plt.title('Histogram of wind speeds for entries with \'tree\' or \'branch\' mentioned in the comments')
plt.xlabel('Wind speed (gust factor = 1.6, m/s)')